## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [16]:
import os
os.getcwd()

'/Users/vss/Personal/Git/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples'

In [18]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [19]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [20]:
mlflow.search_experiments()

[<Experiment: artifact_location='/Users/vss/Personal/Git/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts/0', creation_time=1752045736806, experiment_id='0', last_update_time=1752045736806, lifecycle_stage='active', name='Default', tags={}>]

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/07/09 12:53:23 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/07/09 12:53:26 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 3.1.1, required: mlflow==2.18.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/07/09 12:53:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: '/Users/vss/Personal/Git/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts/1/c74f162dc0fa4983bba322e31b90348b/artifacts'
🏃 View run placid-carp-197 at: http://127.0.0.1:5000/#/experiments/1/runs/c74f162dc0fa4983bba322e31b90348b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [22]:
mlflow.search_experiments()


[<Experiment: artifact_location='/Users/vss/Personal/Git/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts/1', creation_time=1752045803228, experiment_id='1', last_update_time=1752045803228, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/Users/vss/Personal/Git/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts/0', creation_time=1752045736806, experiment_id='0', last_update_time=1752045736806, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [24]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [25]:
client.search_registered_models()

[]

In [26]:
client.search_runs(experiment_ids='1')

[<Run: data=<RunData: metrics={'accuracy': 0.96}, params={'C': '0.1', 'random_state': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "c74f162dc0fa4983bba322e31b90348b", '
                              '"artifact_path": "models", "utc_time_created": '
                              '"2025-07-09 07:23:23.776465", "model_uuid": '
                              '"71980ff778ec49218ca5924b139fef40", "flavors": '
                              '{"python_function": {"model_path": "model.pkl", '
                              '"predict_fn": "predict", "loader_module": '
                              '"mlflow.sklearn", "python_version": "3.12.11", '
                              '"env": {"conda": "conda.yaml", "virtualenv": '
                              '"python_env.yaml"}}, "sklearn": '
                              '{"pickled_model": "model.pkl", '
                              '"sklearn_version": "1.6.1", '
                              '"serialization_format": "cloudpickle", "code": '
 

In [27]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/07/09 12:54:06 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1752045846854, current_stage='None', description='', last_updated_timestamp=1752045846854, name='iris-classifier', run_id='c74f162dc0fa4983bba322e31b90348b', run_link='', source='/Users/vss/Personal/Git/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts/1/c74f162dc0fa4983bba322e31b90348b/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>